## Account Factory setup

In this notebook we will be creating an account factory service using CloudFormation templates and Service Catalog to increase the agility and decrease the time it takes to create accounts in you AWS Organization.

### Initialize Notebook

In [ ]:
import boto3
import botocore
import json
import time
import os

import project_path
from lib import workshop

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
cfn = boto3.client('cloudformation')
lambda_client = boto3.client('lambda')
sc = boto3.client('servicecatalog')

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

### [Create S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)

We will create an S3 bucket that will be used throughout the workshop for storing our data.

[s3.create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket) boto3 documentation

In [ ]:
bucket = workshop.create_bucket(region, session, 'account-')
print(bucket)

### Download Account Factory files for setup

Files for this notebook taken from the [Account Factory](https://github.com/aws-samples/account-factory) repo hosted on [aws-samples](https://github.com/aws-samples/). 

In [ ]:
!wget https://raw.githubusercontent.com/aws-samples/account-factory/master/AccountCreationLambda.py
!wget https://raw.githubusercontent.com/aws-samples/account-factory/master/Accountbaseline.yml
!wget https://raw.githubusercontent.com/aws-samples/account-factory/master/accountbuilder.yml

### View Account Creation script

In [ ]:
!pygmentize AccountCreationLambda.py

### Zip the Account Creation script for use with AWS Lambda

The account creation script will be used when baselining a new account in your AWS Organization. We need to zip the file and upload all files to a known S3 bucket for use later in the notebook.

In [ ]:
%%bash
zip -r AccountCreationLambda.zip AccountCreationLambda.py

### [Upload to S3](https://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html)

Next, we will upload the files needed for the account factory to S3 to be used later in the workshop.

[s3.upload_file](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.upload_file) boto3 documentation

In [ ]:
files = ['Accountbaseline.yml', 'accountbuilder.yml', 'AccountCreationLambda.zip']

for file_name in files:
    session.resource('s3').Bucket(bucket).Object(file_name).upload_file(file_name)

### [Create Service Catalog Portfolio](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/portfoliomgmt-create.html)

[AWS Service Catalog](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/introduction.html) enables organizations to create and manage catalogs of IT services that are approved for use on AWS. These IT services can include everything from virtual machine images, servers, software, and databases to complete multi-tier application architectures. AWS Service Catalog allows organizations to centrally manage commonly deployed IT services, and helps organizations achieve consistent governance and meet compliance requirements. End users can quickly deploy only the approved IT services they need, following the constraints set by your organization.

To provide users with products, begin by creating a portfolio for those products.

[servicecatalog.create_portfolio](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/servicecatalog.html#ServiceCatalog.Client.create_portfolio)

In [ ]:
!cat accountbuilder.yml

In [ ]:
response = sc.create_portfolio(
    AcceptLanguage='en',
    DisplayName='Account Management',
    Description='Portfolio for managing account creation',
    ProviderName='AWS Workshop Owner'
)

portfolio_id = response['PortfolioDetail']['Id']

In [ ]:
print('https://{0}.console.aws.amazon.com/servicecatalog/home?region={0}#/portfolio/details?portfolioId={1}'.format(region, portfolio_id))

### [Create Account Builder Service Catalog Product](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/catalogs_products.html)

You create products by packaging an AWS CloudFormation template with metadata, update products by creating a new version based on an updated template, and group products together into portfolios to distribute them to users. We will be packaging up the `accountbuilder.yml` file as a product to create new accounts with your AWS Organization.

[servicecatalog.create_product](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/servicecatalog.html#ServiceCatalog.Client.create_product)

In [ ]:
template = 'https://s3.amazonaws.com/{0}/{1}'.format(bucket, files[1])
print(template)

response = sc.create_product(
    AcceptLanguage='en',
    Name='Account Builder',
    Owner='AWS Workshop Owner',
    Description='Account Builder product to on-board new accounts',
    Distributor='Central IT',
    SupportDescription='Reach out for any issues to the details below.',
    SupportEmail='admin@example.com',
    SupportUrl='https://support.example.com',
    ProductType='CLOUD_FORMATION_TEMPLATE',
    Tags=[
        {
            'Key': 'Code',
            'Value': 'AS'
        },
    ],
    ProvisioningArtifactParameters={
        'Name': 'v1',
        'Description': 'Version 1 of account builder',
        'Info': {
            'LoadTemplateFromURL': template
        },
        'Type': 'CLOUD_FORMATION_TEMPLATE'
    }
)

product_id = response['ProductViewDetail']['ProductViewSummary']['ProductId']
print(product_id)

### [Add Account Builder product to portfolio](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/catalogs_portfolios_adding-products.html)

To provide users with products, begin by creating a portfolio for those products.

[servicecatalog.associate_product_with_portfolio](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/servicecatalog.html#ServiceCatalog.Client.associate_product_with_portfolio)

In [ ]:
response = sc.associate_product_with_portfolio(
    AcceptLanguage='en',
    ProductId=product_id,
    PortfolioId=portfolio_id
)

print(response)

### [Configure Self Service Access to Portfolios](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/using-service-actions.html)

AWS Service Catalog enables you to reduce administrative maintenance and end-user training while adhering to compliance and security measures. With self-service actions, as the administrator you can enable end users to perform operational tasks, troubleshoot issues, run approved commands, or request permissions in AWS Service Catalog.

We will use the `Admin` group in this example but choose the appropriate role for this portfolio in your own environment.

In [ ]:
iam_group_arn = 'arn:aws:iam::{0}:group/Admin'.format(account_id)

response = sc.associate_principal_with_portfolio(
    AcceptLanguage='en',
    PortfolioId=portfolio_id,
    PrincipalARN=iam_group_arn,
    PrincipalType='IAM'
)

print(response)

### Access the portfolio and launch Account Builder

Clicking on the link below we will launch a new AWS account for the Organization. Use the values below and click `Next` to add the `Parameters` specific to the account you will be creating.

* Name: `TestAccount`
* Version `v1`

On the `Parameters` name fill out the required fields and click `Next`. Here you can add [TagOptions](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/tagoptions.html) and [Notifications](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/constraints-notification.html) and finally click `Launch` to create the new account.

For the `Parameters` section when launching fill out fields with your specific account information. The parameters below give you an example of what to put in a few of the values.

* `stackname`: `Accountbaseline.yml`
* `sourcebucket`: {{bucket created above}}
* `newrole`: You can leave the default or give it a unique name.
* `newrolepolicy`: Free formed text of a valid AWS IAM Policy. The default is `Admin` access. 

In [ ]:
print('https://{0}.console.aws.amazon.com/servicecatalog/home?region={0}#/products'.format(region))

## Clean Up

In [ ]:
response = sc.disassociate_product_from_portfolio(
    AcceptLanguage='en',
    ProductId=product_id,
    PortfolioId=portfolio_id
)

In [ ]:
response = sc.delete_product(
    AcceptLanguage='en',
    Id=product_id
)

In [ ]:
response = sc.disassociate_principal_from_portfolio(
    AcceptLanguage='en',
    PortfolioId=portfolio_id,
    PrincipalARN=iam_group_arn
)

In [ ]:
response = sc.delete_portfolio(
    AcceptLanguage='en',
    Id=portfolio_id
)

In [ ]:
workshop.delete_bucket_completely(bucket)